In [26]:
"""
Sources used:
https://www.kaggle.com/datasets/phsophea101/image-for-stitching
https://www.geeksforgeeks.org/python/finding-the-size-resolution-of-image-in-python/
https://www.geeksforgeeks.org/computer-vision/how-to-check-for-blurry-images-in-your-dataset-using-the-laplacian-method/
https://stackoverflow.com/questions/25524192/how-to-get-the-signal-to-noise-ratio-from-an-image-in-python
https://github.com/opencv/opencv_contrib/tree/master/modules/quality/samples
https://raw.githubusercontent.com/opencv/opencv_contrib/master/modules/quality/samples/brisque_model_live.yml
https://raw.githubusercontent.com/opencv/opencv_contrib/master/modules/quality/samples/brisque_range_live.yml
https://stackoverflow.com/questions/58821130/how-to-calculate-the-contrast-of-an-image
https://stackoverflow.com/questions/50313114/what-is-the-entropy-of-an-image-and-how-is-it-calculated
https://stackoverflow.com/questions/58831690/how-to-measure-the-saturation-of-an-image
https://stackoverflow.com/questions/55405639/extract-hue-channel-from-hsv-image-in-pillow
https://stackoverflow.com/questions/8960462/how-can-i-measure-image-noise
"""

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
import skimage.measure
import pandas as pd

folder = 'images/original/'
image_files = [f for f in os.listdir(folder) if f.endswith(('.png', '.jpg'))]

images = []
names = []
for f in sorted(image_files)[:1000]:
    path = os.path.join(folder, f)
    img = cv2.imread(path)
    if img is not None and len(img.shape) == 3:
        images.append(img)
        names.append(f)

print(f"{len(images)} images uploaded")

1000 images uploaded


In [27]:
# resolution
def resuliton(image):
    wid = images[0].shape[1]
    hgt = images[0].shape[0]
    #print(str(wid) + "x" + str(hgt))

In [28]:
# blur level
def blur_level(image):
    """
    applies laplacian filter and takes the variance of the image
    """
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    var_lap = laplacian.var()
    return var_lap

#blr = blur_level(images[0])
#print(blr)

In [29]:
# signal-to-noise-ratio
def signal_to_noise(arr, axis=0, ddof=0):
    """
    use .ravel() fo a single value
    """
    arr = np.asanyarray(arr)
    mean = arr.mean(axis)
    std = arr.std(axis=axis, ddof=ddof)
    return np.where(std == 0, 0, mean / std)
#snr = signal_to_noise(images[0].ravel())
#print(snr)

In [30]:
# luminance
def mean_luminance(image):
    if image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    R = image[:, :, 0]
    G = image[:, :, 1]
    B = image[:, :, 2]
    # Luminance
    Y = 0.299 * R + 0.587 * G + 0.114 * B
    return np.mean(Y)
#lum = mean_luminance(images[0])
#print(lum)

In [42]:
# quality
def quality_brisque(image):
    if len(image.shape) == 2:
        img = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    else:
        img = image.copy()
    brisque = cv2.quality.QualityBRISQUE_create(
        "brisque_model_live.yml",
        "brisque_range_live.yml"
    )
    score = brisque.compute(img)
    
    if isinstance(score, tuple) or isinstance(score, list):
        return score[0]
    else:
        return score

#qual_score = quality_brisque(images[0])
#print("BRISQUE score:", qual_score)

In [32]:
#Had some versioning problems...
#print(cv2.__version__) 
#print(hasattr(cv2, "quality"))

In [33]:
# contrast
def contrast(image):
    img_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contrast = img_grey.std()

    return contrast

#crst = contrast(images[0])
#print(crst)
    

In [34]:
# entropy
def entropy(image):    
    entropy = skimage.measure.shannon_entropy(image)
    return entropy

#trp = entropy(images[0])
#print(etrp)

In [43]:
def saturation(image):
    if image is None or len(image.shape) != 3 or image.shape[2] != 3:
        return None
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    return img_hsv[:, :, 1].mean()

In [36]:
# mean hue
def mean_hue(image):
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mean_hue_value = img_hsv[:, :, 0].mean()
    return mean_hue_value

#hue_val = mean_hue(images[0])
#print(hue_val)

In [37]:
def estimate_noise(image):
    # Convert to grayscale
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    # Apply a Gaussian blur to get the smooth background
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    # The noise residual
    noise = gray.astype(np.float32) - blur.astype(np.float32)
    # The standard deviation is an estimate of the noise level
    noise_std = noise.std()
    return noise_std


#noise_lvl = estimate_noise(images[0])
#print(noise_lvl)

In [44]:
data = []

for idx, img in enumerate(images): 
    row = {}
    row['Name'] = f"image_{idx+1:04d}"

    # BRISQUE
    try:
        row['BRISQUE Score'] = float(quality_brisque(img))
    except Exception as e:
        row['BRISQUE Score'] = None

    # Blur Level
    try:
        row['Blur Level'] = float(blur_level(img))
    except Exception as e:
        row['Blur Level'] = None

    # Mean Luminance
    try:
        row['Mean Luminance'] = mean_luminance(img)
    except Exception as e:
        row['Mean Luminance'] = None

    # Contrast
    try:
        row['Contrast'] = contrast(img)
    except Exception as e:
        row['Contrast'] = None

    # Entropy
    try:
        row['Entropy'] = entropy(img)
    except Exception as e:
        row['Entropy'] = None

    # Mean Saturation
    try:
        row['Mean Saturation'] = saturation(img)
    except Exception as e:
        row['Mean Saturation'] = None

    # Mean Hue
    try:
        row['Mean Hue'] = mean_hue(img)
    except Exception as e:
        row['Mean Hue'] = None

    # Noise Level
    try:
        row['Noise Level'] = estimate_noise(img)
    except Exception as e:
        row['Noise Level'] = None

    # (Optional) Image Size
    try:
        row['Height'], row['Width'] = img.shape[:2]
    except Exception as e:
        row['Height'], row['Width'] = None, None

    data.append(row)


df = pd.DataFrame(data)

df.head()

,Name,BRISQUE Score,Blur Level,Mean Luminance,Contrast,Entropy,Mean Saturation,Mean Hue,Noise Level,Height,Width
0,image_0001,31.658161,1634.948492,127.282674,57.401085,7.595879,56.236417,74.421475,15.575411,1080,1920
1,image_0002,27.825500,1519.258310,126.920166,57.741467,7.597329,56.434439,74.362521,15.428082,1080,1920
2,image_0003,27.576078,1506.326255,126.300778,57.695397,7.596898,56.045522,73.079647,15.271338,1080,1920
3,image_0004,28.331348,1443.249310,125.894826,57.608786,7.582865,56.740936,73.030836,15.077243,1080,1920
4,image_0005,32.038090,1520.450882,125.883328,57.616889,7.586458,59.733074,75.967386,15.197158,1080,1920


In [45]:
df = df.round(2)
df.head()

,Name,BRISQUE Score,Blur Level,Mean Luminance,Contrast,Entropy,Mean Saturation,Mean Hue,Noise Level,Height,Width
0,image_0001,31.66,1634.95,127.28,57.40,7.60,56.24,74.42,15.58,1080,1920
1,image_0002,27.83,1519.26,126.92,57.74,7.60,56.43,74.36,15.43,1080,1920
2,image_0003,27.58,1506.33,126.30,57.70,7.60,56.05,73.08,15.27,1080,1920
3,image_0004,28.33,1443.25,125.89,57.61,7.58,56.74,73.03,15.08,1080,1920
4,image_0005,32.04,1520.45,125.88,57.62,7.59,59.73,75.97,15.20,1080,1920


In [46]:
df['Noise Level'] = df['Noise Level'].astype(float).round(2)
df.tail().round(2)

,Name,BRISQUE Score,Blur Level,Mean Luminance,Contrast,Entropy,Mean Saturation,Mean Hue,Noise Level,Height,Width
995,image_0996,30.82,3068.49,120.39,59.58,7.84,57.66,50.08,20.61,1080,1920
996,image_0997,31.32,3021.82,120.61,61.26,7.84,58.59,49.14,20.35,1080,1920
997,image_0998,33.62,3062.32,116.97,62.50,7.82,60.90,47.27,20.39,1080,1920
998,image_0999,32.54,3107.96,118.47,62.44,7.84,60.42,46.58,20.45,1080,1920
999,image_1000,31.09,3308.38,122.91,62.60,7.88,58.79,46.78,21.03,1080,1920


In [47]:
df.describe().round(2)

,BRISQUE Score,Blur Level,Mean Luminance,Contrast,Entropy,Mean Saturation,Mean Hue,Noise Level,Height,Width
count,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.0
mean,27.01,1794.89,122.85,57.80,7.69,65.64,63.50,16.20,1080.0,1920.0
std,7.57,885.56,8.89,8.59,0.20,11.97,12.85,3.54,0.0,0.0
min,10.29,311.60,81.18,29.91,6.51,23.73,28.11,7.79,1080.0,1920.0
25%,20.91,1028.50,116.78,54.31,7.63,58.40,54.16,13.31,1080.0,1920.0
50%,25.96,1626.14,122.98,58.84,7.75,65.84,63.36,16.20,1080.0,1920.0
75%,32.19,2541.19,128.97,63.38,7.82,72.74,71.00,19.23,1080.0,1920.0
max,52.26,4112.79,151.47,76.59,7.93,112.49,105.02,23.64,1080.0,1920.0
